In [1]:
import fmlib
import numpy as np
import scipy.optimize
from grading_tools import auto_marking_message

# Exercises

## Exercise

Find a root of the equation $x^7 + 5 x^3 - 3 x + 1 =0$. Store the answer in a variable called `x`.

In [2]:
### BEGIN SOLUTION
def f( x ):
    return x**7 + 5*x**3 - 3*x + 1
sol = scipy.optimize.root_scalar(f, x0=-1.0, x1=1.0, method='secant')
assert sol.converged
x = sol.root
### END SOLUTION

In [3]:
np.testing.assert_almost_equal( x**7 + 5*x**3 - 3*x + 1, 0, decimal=7)
auto_marking_message()

Auto marking message: 😊 Correct


## Exercise

A European put option has strike $K=110$ and maturity $T=0.5$.
The current stock price is $S_0=100$, the risk-free rate is $r=0.02$. The price
of the option is 11.8. What is the implied volatility? Store your answer
in a variable called `implied_vol`

In [4]:
K = 110
T = 0.5
S0 = 100
r = 0.02
sigma = 0.2
put_option_price = 11.8
### BEGIN SOLUTION
V = fmlib.black_scholes_put_price(S0,0,K,T,r,sigma)

def compute_implied_volatility_put( V_KT, S0, K, T, r ):
    def f( sigma ):
        return fmlib.black_scholes_put_price(S0,0,K,T,r,sigma)-V_KT
    sol = scipy.optimize.root_scalar(f, x0=0.01, x1=1.0, method='secant')
    assert sol.converged
    return sol.root

implied_vol = compute_implied_volatility_put(put_option_price, S0, K, T, r )
### END SOLUTION

In [5]:
np.testing.assert_almost_equal( fmlib.black_scholes_put_price(S0,0,K,T,r,implied_vol), put_option_price, decimal=6 )
auto_marking_message()

Auto marking message: 🌟 Correct


## Exercise

Explain the difference between:
    
 * The *implied volatility* of a call option
 * The volatility of the stock price in the Black-Scholes model
 * The *historic volatility* of the stock price

Under what theoretical assumptions would these quantities be the same? Are these assumptions plausible in a real market?

The *implied volatility* is the value of $\sigma_{K,T}$ that solves
$$
V(S,K,T,r,\sigma_{K,T})=V_{K,T}
$$
where $V_{K,T}$ is the price of the call option and $V(S,K,T,r,\sigma_{K,T})$ is the Black-Scholes call price formula.
It is a property of the option price and will typically vary with the strike $K$ and the maturity $T$.

The *volatility* of the stock price in the Black-Scholes model is one of the parameters which determines
the future evolution of the stock price. It is a property of our probability model for the future.

The *historic volatility* is a statistical measure of the historical variability of the stock price. There are numerous ways
of measuring this, so the term *historic volatility* isn't very clearly defined. For example, one could take the standard deviation of
the daily changes in the log of the stock price over a year and multiply by $(365)^{\frac{1}{2}}$ to obtain one possible measure
of the historic volatility. It is a property if

If the stock price has always followed geometric Brownian motion and it is known that it always will, and if all the other assumptions
of the Black-Scholes model are true, then these quantities would be the same. In practice this will never happen.



## Remark

The call options we have considered on Google stock are in fact American options not European options. Typically options on an individual stock will be American options whereas options on an index are European options.

Google options do not pay dividends. In fact, throughout this course we have only considered stocks that do not pay dividends. (To apply the theory we have considered to stocks which pay dividends is to assume that one reinvests any dividends paid by a company into the stock of that company. One can then model the value of the asset "Stock + dividend" rather than model the stock price directly to obtain similar results.)

American call options and European call options are equivalent for options which do not pay dividends if the interest rate $r\geq 0$.

To see this
it suffices to prove that at time $t$, a European call option with strike $K$ and maturity $T$ is always worth more than $\max\{S_t - K, 0\}$, as this implies
that there is no benefit to exercising an American option before maturity.

To prove our claim we use a replication argument:
* If you have $S_t-K$ cash at time $t$, you could purchase one unit of stock and put $-K$ in the risk-free account. At time $T$ this will be worth $S_T-e^{r(T-t)}K$. So having $S_t-K$ cash at time $t$ allows you to replicate $S_T-e^{r(T-t)}K$. Hence the price of a derivative with payoff
$S_T-e^{r(T-t)}K$ must be $S_t-K$.
* But $\max\{S_T-K,0\} \geq S_T - K \geq S_T - e^{r(T-t)}K$ since $r \geq 0$. So a European call option with strike $K$ and maturity $T$ is always worth more than a derivative with payoff $S_T - e^{r(T-t)}K$, and so it is always worth more than $S_t-K$.
* Since the payoff of a European call option is always non-negative, European call options must be worth at least $0$ too.
* Hence at time $t$, a European call option with strike $K$ and maturity $T$ is always worth more than $\max\{S_t-K,0\}$ as claimed.

This means that you can price American call options in the Black-Scholes model using the Black-Scholes formula. However, American put options are trickier to price in the Black-Scholes model.